In [11]:
import requests
import pandas as pd
import config
import enum

In [7]:
def get_match_details(match_id):
    root_url = "https://api.steampowered.com"
    command_type = "/IDOTA2Match_570"
    command = "/GetMatchDetails/v1"

    url = root_url + command_type + command
    api_key = config.STEAM_API_KEY
    query_params = {
        "key": api_key,
        "partner": 0,
        "match_id": match_id,
        "include_persona_names": 1,
    }

    res = requests.get(url, query_params)

    if res.ok:
        data = res.json()
        if not "error" in data["result"].keys():

            # with open('match.json', 'w') as f:
            #     json.dump(data, f, indent=4)
            #     f.close()

            return data

    return None

### Get player's matches by his steam_id by using Dota Web API

In [8]:
def get_player_matches(account_id, game_mode, match_requested):
    root_url = "https://api.steampowered.com"
    command_type = "/IDOTA2Match_570"
    command = "/GetMatchHistory/v1"

    url = root_url + command_type + command
    api_key = config.STEAM_API_KEY
    query_params = {
        "key": api_key,
        "partner": 0,
        "account_id": account_id,
        "matches_requested": match_requested,
        "game_mode": game_mode,
    }

    res = requests.get(url, query_params)

    if res.ok:
        data = res.json()

        # with open(os.getcwd() + '/data/playerStatsInMatches.json', 'w') as f:
        #     json.dump(data, f, indent=4)
        #     f.close()

        return data

    return None

In [9]:
def get_player_match_stats(match_id, account_id):
    match = get_match_details(match_id)
    player_stats = {
        "persona": "",
        "team_number": 0,
        "kills": 0,
        "deaths": 0,
        "assists": 0,
        "last_hits": 254,
        "denies": 0,
        "gold_per_min": 0,
        "xp_per_min": 0,
        "level": 0,
        "net_worth": 0,
        "hero_damage": 0,
        "tower_damage": 0,
        "hero_healing": 0,
        "gold_spent": 0,
        "scaled_hero_damage": 0,
        "scaled_tower_damage": 0,
        "scaled_hero_healing": 0,
    }

    if match is not None:
        for player in match["result"]["players"]:
            try:
                if player["account_id"] == account_id:
                    for stat in player_stats.keys():
                        player_stats[stat] = player[stat]
            except Exception as e:
                continue

        #!print(player_stats)

        player_stats["win"] = (
            1
            if (
                not player_stats["team_number"]
                and match["result"]["radiant_win"]
                or player_stats["team_number"]
                and not match["result"]["radiant_win"]
            )
            else 0
        )
        # print('match:', match['result']['match_id'], 'team_number', player_stats['team_number'], 'radiant_win:',
        #       match['result']['radiant_win'], 'win:', player_stats['win'])

        return player_stats

    return None

### Get one player statistics

In [10]:
def process_player_data(account_id, game_mode, match_requested):
    data = get_player_matches(account_id, game_mode, match_requested)
    name = ""
    middle_stats = {
        "win": 0,
        "team_number": 0,
        "kills": 0,
        "deaths": 0,
        "assists": 0,
        "last_hits": 254,
        "denies": 0,
        "gold_per_min": 0,
        "xp_per_min": 0,
        "level": 0,
        "net_worth": 0,
        "hero_damage": 0,
        "tower_damage": 0,
        "hero_healing": 0,
        "gold_spent": 0,
        "scaled_hero_damage": 0,
        "scaled_tower_damage": 0,
        "scaled_hero_healing": 0,
    }

    if data is not None and "matches" in data["result"].keys():
        for match in data["result"]["matches"]:
            #!print(match['match_id'])
            match_stats = get_player_match_stats(match["match_id"], account_id)

            if match is not None:
                name = match_stats["persona"]
                del match_stats["persona"]
                for stat in match_stats.keys():
                    middle_stats[stat] += match_stats[stat]

        for stat in middle_stats.keys():
            middle_stats[stat] /= len(data["result"]["matches"])

        middle_stats["team_number"] = 1 - middle_stats["team_number"]
        middle_stats["matches"] = len(data["result"]["matches"])
        middle_stats["radiant"] = middle_stats["team_number"]
        middle_stats["name"] = name
        middle_stats["account_id"] = account_id

        del middle_stats["team_number"]

        return middle_stats

    return None

In [12]:
class GameMode(enum.Enum):
    NO_MODE = 0
    ALL_PICK = 1
    CAP_MODE = 2
    RANDOM_DRAFT = 3
    SINGLE_DRAFT = 4

####   Get player's with steam_id = 178749178 info about last 20 matches in all pick

In [22]:
data = process_player_data(178749178, GameMode.ALL_PICK, 20)

In [24]:
def json2DF(json_data):
    return pd.json_normalize(json_data)

### Get stats top 10 world players

In [26]:
def get_players_df_stats(player_ids : list[int], matches_count : int, game_mode : GameMode) -> pd.DataFrame:
    json_data = []

    for id in player_ids:
        json_data.append(process_player_data(account_id=id, game_mode=game_mode, match_requested=matches_count))

    return json2DF(json_data)

In [30]:
df_players = get_players_df_stats(
    [
        171262902,
        93618577,
        321580662,
        73401082,
        898455820,
        183719386,
        56351509,
        480412663,
        1044002267,
        302214028,
    ],
    20,
    GameMode.ALL_PICK,
)
df_players

,win,kills,deaths,assists,last_hits,denies,gold_per_min,xp_per_min,level,net_worth,...,tower_damage,hero_healing,gold_spent,scaled_hero_damage,scaled_tower_damage,scaled_hero_healing,matches,radiant,name,account_id
0,0.60,9.90,3.10,9.55,299.05,14.65,796.20,943.00,21.05,22007.20,...,9317.15,259.50,20164.50,15082.95,4219.85,109.30,20,0.50,1,171262902
1,0.75,10.00,5.05,12.10,263.10,12.30,680.85,821.20,20.80,20533.30,...,4479.50,112.40,20012.75,17902.45,2101.55,60.45,20,0.40,ammar is my goat,93618577
2,0.55,12.20,5.20,11.05,372.00,9.00,700.70,889.55,23.75,26746.10,...,7927.50,1972.45,27032.50,17882.60,3522.30,536.20,20,0.25,yao,321580662
3,0.65,2.75,5.65,17.45,79.45,2.25,345.90,464.55,16.75,10917.65,...,3037.35,6238.85,11026.75,9095.65,1389.60,1680.10,20,0.65,Dukalis,73401082
4,0.60,9.05,4.55,10.00,319.70,14.40,636.65,731.55,20.85,20863.50,...,6689.55,0.00,20383.25,16757.55,3045.25,0.00,20,0.55,Буська,898455820
5,0.55,9.00,4.90,11.75,308.40,14.65,625.50,755.90,21.30,21772.05,...,4366.10,101.15,22914.25,16293.35,2183.00,29.10,20,0.45,Suma1L-,183719386
6,0.50,6.50,4.70,17.55,283.10,11.70,566.20,680.85,21.40,20008.60,...,2898.85,1012.70,19555.50,13576.55,1322.05,358.15,20,0.40,Offlane Bobi,56351509
7,0.40,8.70,4.70,10.60,348.20,12.35,639.65,748.25,21.40,21499.25,...,3069.40,832.20,21624.50,18093.15,1423.10,340.25,20,0.55,yuuuuuuuuuuuuuuuuuuuuuuuuuuuu,480412663
8,0.60,9.85,3.05,8.95,432.00,10.90,737.45,844.95,22.25,26462.55,...,9068.70,47.40,26494.75,15169.05,4025.90,21.95,20,0.60,4,1044002267
9,0.65,6.70,3.85,13.30,248.10,11.80,565.85,675.60,19.90,19240.80,...,1851.30,908.05,17837.50,11220.30,796.95,358.85,20,0.75,cao,302214028


In [31]:
df_players.to_csv('top10.csv')

In [32]:
1111111

1111